## Objective is to find the Mobile Handsets similar to the given Mobile Handset based on features such as network, 3G, 4G bands , battery , price ,sensors etc.

In [101]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [102]:
from mlxtend.preprocessing import TransactionEncoder

In [103]:
df = pd.read_csv("HandsetData.csv")

In [104]:
df.head()


,Unnamed: 0,brand,model,network_technology,2G_bands,3G_bands,4G_bands,network_speed,GPRS,EDGE,...,bluetooth,GPS,NFC,radio,USB,sensors,battery,colors,approx_price_EUR,img_url
0,0,Apple,iPad Pro 12.9,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A (3CA) Cat9 450/50 M...,Yes,Yes,...,Yes,Yes,Yes,No,microUSB 3.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Space Gray| Gold| Silver,Super High,http://cdn2.gsmarena.com/vv/bigpic/apple-ipad-...
1,1,Apple,iPad Pro 10.5,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A (3CA) Cat9 450/50 M...,Yes,Yes,...,Yes,Yes,Yes,No,microUSB 3.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Space Gray| Rose Gold| Gold| Silver,Super High,http://cdn2.gsmarena.com/vv/bigpic/apple-ipad-...
2,2,Apple,iPad 9.7,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A Cat4 150/50 Mbps E...,Yes,Yes,...,Yes,Yes,No,No,microUSB 2.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Silver| Gold| Space Gray,High,http://cdn2.gsmarena.com/vv/bigpic/apple-ipad-...
3,3,Huawei,Honor 9,GSM / CDMA / HSPA / LTE,1,1,1,HSPA LTE,Yes,Yes,...,Yes,Yes,Yes,No,Type C 1.0,Fingerprint (front-mounted)| accelerometer| gy...,High,Amber Gold| Seagull Grey| Charm Sea Blue,High,http://cdn2.gsmarena.com/vv/bigpic/huawei-hono...
4,4,Huawei,Y7 Prime,GSM / CDMA / HSPA / LTE,1,1,1,HSPA LTE,Yes,Yes,...,Yes,Yes,No,Yes,microUSB 2.0,Fingerprint (rear-mounted)| accelerometer| pro...,High,Silver| gray| gold,Moderate,http://cdn2.gsmarena.com/vv/bigpic/huawei-enjo...


## Below we are selecting relevant features

In [105]:
df.columns

Index(['Unnamed: 0', 'brand', 'model', 'network_technology', '2G_bands',
       '3G_bands', '4G_bands', 'network_speed', 'GPRS', 'EDGE', 'announced',
       'dimentions', 'weight_g', 'SIM', 'display_type', 'display_resolution',
       'display_size', 'OS', 'CPU', 'GPU', 'memory_card', 'internal_memory',
       'RAM', 'primary_camera', 'secondary_camera', 'loud_speaker',
       'audio_jack', 'WLAN', 'bluetooth', 'GPS', 'NFC', 'radio', 'USB',
       'sensors', 'battery', 'colors', 'approx_price_EUR', 'img_url'],
      dtype='object')

In [106]:
df1 = df[['model', 'network_technology', '2G_bands',
       '3G_bands', '4G_bands', 'network_speed', 'GPRS', 'EDGE', 'announced',
       'dimentions', 'weight_g', 'SIM', 'display_type', 'display_resolution',
       'display_size', 'OS', 'CPU', 'GPU', 'memory_card', 'internal_memory',
       'RAM', 'primary_camera', 'secondary_camera', 'loud_speaker',
       'audio_jack', 'WLAN', 'bluetooth', 'GPS', 'NFC', 'radio', 'USB',
       'sensors', 'battery', 'colors', 'approx_price_EUR',]]

In [107]:
df1.head()

,model,network_technology,2G_bands,3G_bands,4G_bands,network_speed,GPRS,EDGE,announced,dimentions,...,WLAN,bluetooth,GPS,NFC,radio,USB,sensors,battery,colors,approx_price_EUR
0,iPad Pro 12.9,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A (3CA) Cat9 450/50 M...,Yes,Yes,2017,305.7 x 220.6 x 6.9 mm (12.04 x 8.69 x 0.27 in),...,Yes,Yes,Yes,Yes,No,microUSB 3.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Space Gray| Gold| Silver,Super High
1,iPad Pro 10.5,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A (3CA) Cat9 450/50 M...,Yes,Yes,2017,250.6 x 174.1 x 6.1 mm (9.87 x 6.85 x 0.24 in),...,Yes,Yes,Yes,Yes,No,microUSB 3.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Space Gray| Rose Gold| Gold| Silver,Super High
2,iPad 9.7,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A Cat4 150/50 Mbps E...,Yes,Yes,2017,240 x 169.5 x 7.5 mm (9.45 x 6.67 x 0.30 in),...,Yes,Yes,Yes,No,No,microUSB 2.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Silver| Gold| Space Gray,High
3,Honor 9,GSM / CDMA / HSPA / LTE,1,1,1,HSPA LTE,Yes,Yes,2017,147.3 x 70.9 x 7.5 mm (5.80 x 2.79 x 0.30 in),...,Yes,Yes,Yes,Yes,No,Type C 1.0,Fingerprint (front-mounted)| accelerometer| gy...,High,Amber Gold| Seagull Grey| Charm Sea Blue,High
4,Y7 Prime,GSM / CDMA / HSPA / LTE,1,1,1,HSPA LTE,Yes,Yes,2017,153.6 x 76.4 x 8.4 mm (6.05 x 3.01 x 0.33 in),...,Yes,Yes,Yes,No,Yes,microUSB 2.0,Fingerprint (rear-mounted)| accelerometer| pro...,High,Silver| gray| gold,Moderate


## combined all features against the model column

In [108]:
l1 = df1.iloc[1][1:].values.tolist()

In [109]:
df2 = df1.applymap(str).copy()

In [110]:
df2['combined'] = [df2.iloc[i][1:].values.tolist() for i in range(0,df2.shape[0])]

In [111]:
df2.head()

,model,network_technology,2G_bands,3G_bands,4G_bands,network_speed,GPRS,EDGE,announced,dimentions,...,bluetooth,GPS,NFC,radio,USB,sensors,battery,colors,approx_price_EUR,combined
0,iPad Pro 12.9,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A (3CA) Cat9 450/50 M...,Yes,Yes,2017,305.7 x 220.6 x 6.9 mm (12.04 x 8.69 x 0.27 in),...,Yes,Yes,Yes,No,microUSB 3.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Space Gray| Gold| Silver,Super High,"[GSM / CDMA / HSPA / EVDO / LTE, 1, 1, 1, HSPA..."
1,iPad Pro 10.5,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A (3CA) Cat9 450/50 M...,Yes,Yes,2017,250.6 x 174.1 x 6.1 mm (9.87 x 6.85 x 0.24 in),...,Yes,Yes,Yes,No,microUSB 3.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Space Gray| Rose Gold| Gold| Silver,Super High,"[GSM / CDMA / HSPA / EVDO / LTE, 1, 1, 1, HSPA..."
2,iPad 9.7,GSM / CDMA / HSPA / EVDO / LTE,1,1,1,HSPA 42.2/5.76 Mbps LTE-A Cat4 150/50 Mbps E...,Yes,Yes,2017,240 x 169.5 x 7.5 mm (9.45 x 6.67 x 0.30 in),...,Yes,Yes,No,No,microUSB 2.0,Fingerprint (front-mounted)| accelerometer| gy...,Super High,Silver| Gold| Space Gray,High,"[GSM / CDMA / HSPA / EVDO / LTE, 1, 1, 1, HSPA..."
3,Honor 9,GSM / CDMA / HSPA / LTE,1,1,1,HSPA LTE,Yes,Yes,2017,147.3 x 70.9 x 7.5 mm (5.80 x 2.79 x 0.30 in),...,Yes,Yes,Yes,No,Type C 1.0,Fingerprint (front-mounted)| accelerometer| gy...,High,Amber Gold| Seagull Grey| Charm Sea Blue,High,"[GSM / CDMA / HSPA / LTE, 1, 1, 1, HSPA LTE, ..."
4,Y7 Prime,GSM / CDMA / HSPA / LTE,1,1,1,HSPA LTE,Yes,Yes,2017,153.6 x 76.4 x 8.4 mm (6.05 x 3.01 x 0.33 in),...,Yes,Yes,No,Yes,microUSB 2.0,Fingerprint (rear-mounted)| accelerometer| pro...,High,Silver| gray| gold,Moderate,"[GSM / CDMA / HSPA / LTE, 1, 1, 1, HSPA LTE, ..."


In [112]:
df3 = df2[['model','combined']]

In [113]:
df3.head()

,model,combined
0,iPad Pro 12.9,"[GSM / CDMA / HSPA / EVDO / LTE, 1, 1, 1, HSPA..."
1,iPad Pro 10.5,"[GSM / CDMA / HSPA / EVDO / LTE, 1, 1, 1, HSPA..."
2,iPad 9.7,"[GSM / CDMA / HSPA / EVDO / LTE, 1, 1, 1, HSPA..."
3,Honor 9,"[GSM / CDMA / HSPA / LTE, 1, 1, 1, HSPA LTE, ..."
4,Y7 Prime,"[GSM / CDMA / HSPA / LTE, 1, 1, 1, HSPA LTE, ..."


## Applied Transaction encoder in the data and preparing it for further processing

In [114]:
dataset=list(df3['combined'])


In [67]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

df5 = pd.DataFrame(te_ary, columns=te.columns_)


In [68]:
df5

,1 GB RAM,.5 GB,0,0 GB RAM,0 MP,0 Mp,0.9 MP| 720p,1,1 GB RAM,1 GB RAM - Galaxy J1 Nxt,...,microSD up to 32 GB (dedicated slot),microSD up to 32 GB (dedicated slot) 4 GB included,microSD up to 64 GB (dedicated slot),microSD up to 64 GB (dedicated slot) - single-SIM model,microSD up to 64 GB (uses SIM 2 slot),microSD (dedicated slot),microUSB 2.0,microUSB 3.0,watchOS 1.0| upgradable to 3.2.2,watchOS 3.0| upgradable to 3.2.2
0,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
5,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
6,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
7,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
8,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
9,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False


In [69]:
df6 = df5.astype("int")

In [70]:
df6

,1 GB RAM,.5 GB,0,0 GB RAM,0 MP,0 Mp,0.9 MP| 720p,1,1 GB RAM,1 GB RAM - Galaxy J1 Nxt,...,microSD up to 32 GB (dedicated slot),microSD up to 32 GB (dedicated slot) 4 GB included,microSD up to 64 GB (dedicated slot),microSD up to 64 GB (dedicated slot) - single-SIM model,microSD up to 64 GB (uses SIM 2 slot),microSD (dedicated slot),microUSB 2.0,microUSB 3.0,watchOS 1.0| upgradable to 3.2.2,watchOS 3.0| upgradable to 3.2.2
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [71]:
cl1 = list(df3['model'].values)

In [72]:
len(cl1)

339

## added model column to dataframe

In [73]:
df6['models'] = cl1

In [74]:
df7 = df6.set_index('models')

In [75]:
df7.head()

,1 GB RAM,.5 GB,0,0 GB RAM,0 MP,0 Mp,0.9 MP| 720p,1,1 GB RAM,1 GB RAM - Galaxy J1 Nxt,...,microSD up to 32 GB (dedicated slot),microSD up to 32 GB (dedicated slot) 4 GB included,microSD up to 64 GB (dedicated slot),microSD up to 64 GB (dedicated slot) - single-SIM model,microSD up to 64 GB (uses SIM 2 slot),microSD (dedicated slot),microUSB 2.0,microUSB 3.0,watchOS 1.0| upgradable to 3.2.2,watchOS 3.0| upgradable to 3.2.2
models,,,,,,,,,,,,,,,,,,,,,
iPad Pro 12.9,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
iPad Pro 10.5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
iPad 9.7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
Honor 9,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Y7 Prime,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


## calculated model similarity using cosine similarity

In [76]:
model_similarity = cosine_similarity(df7)

In [77]:
model_sim = pd.DataFrame(model_similarity)

In [78]:
model_sim

,0,1,2,3,4,5,6,7,8,9,...,329,330,331,332,333,334,335,336,337,338
0,1.000000,0.791667,0.408248,0.166667,0.163299,0.244949,0.208333,0.285774,0.166667,0.166667,...,0.244949,0.244949,0.204124,0.240192,0.240192,0.204124,0.200160,0.163299,0.163299,0.122474
1,0.791667,1.000000,0.408248,0.166667,0.163299,0.244949,0.208333,0.285774,0.166667,0.166667,...,0.244949,0.244949,0.204124,0.240192,0.240192,0.204124,0.200160,0.163299,0.163299,0.122474
2,0.408248,0.408248,1.000000,0.204124,0.240000,0.320000,0.285774,0.320000,0.244949,0.244949,...,0.240000,0.240000,0.240000,0.274563,0.274563,0.240000,0.353009,0.320000,0.320000,0.280000
3,0.166667,0.166667,0.204124,1.000000,0.489898,0.285774,0.250000,0.244949,0.458333,0.458333,...,0.122474,0.122474,0.122474,0.160128,0.160128,0.122474,0.160128,0.122474,0.163299,0.122474
4,0.163299,0.163299,0.240000,0.489898,1.000000,0.440000,0.367423,0.400000,0.449073,0.408248,...,0.160000,0.160000,0.160000,0.196116,0.196116,0.160000,0.196116,0.160000,0.200000,0.200000
5,0.244949,0.244949,0.320000,0.285774,0.440000,1.000000,0.408248,0.680000,0.285774,0.285774,...,0.280000,0.240000,0.200000,0.313786,0.274563,0.200000,0.353009,0.240000,0.280000,0.200000
6,0.208333,0.208333,0.285774,0.250000,0.367423,0.408248,1.000000,0.489898,0.333333,0.333333,...,0.285774,0.204124,0.163299,0.320256,0.240192,0.163299,0.360288,0.204124,0.326599,0.244949
7,0.285774,0.285774,0.320000,0.244949,0.400000,0.680000,0.489898,1.000000,0.326599,0.326599,...,0.320000,0.240000,0.200000,0.353009,0.274563,0.200000,0.431455,0.280000,0.360000,0.240000
8,0.166667,0.166667,0.244949,0.458333,0.449073,0.285774,0.333333,0.326599,1.000000,0.791667,...,0.163299,0.122474,0.122474,0.200160,0.160128,0.122474,0.240192,0.163299,0.244949,0.163299
9,0.166667,0.166667,0.244949,0.458333,0.408248,0.285774,0.333333,0.326599,0.791667,1.000000,...,0.163299,0.122474,0.122474,0.200160,0.160128,0.122474,0.240192,0.163299,0.244949,0.163299


In [79]:
data = model_sim.values

In [80]:
final_mod = pd.DataFrame(data, columns = cl1 )

In [81]:
final_mod.head()

,iPad Pro 12.9,iPad Pro 10.5,iPad 9.7,Honor 9,Y7 Prime,MediaPad M3 Lite 8,MediaPad T3 10,MediaPad M3 Lite 10,nova 2 plus,nova 2,...,Galaxy Note Pro 12.2 LTE,Galaxy Note Pro 12.2 3G,Galaxy Note Pro 12.2,Galaxy Tab Pro 12.2 LTE,Galaxy Tab Pro 12.2 3G,Galaxy Tab Pro 12.2,Galaxy Tab Pro 10.1 LTE,Galaxy Tab Pro 10.1,Galaxy Tab Pro 8.4 3G/LTE,Galaxy Tab Pro 8.4
0,1.000000,0.791667,0.408248,0.166667,0.163299,0.244949,0.208333,0.285774,0.166667,0.166667,...,0.244949,0.244949,0.204124,0.240192,0.240192,0.204124,0.200160,0.163299,0.163299,0.122474
1,0.791667,1.000000,0.408248,0.166667,0.163299,0.244949,0.208333,0.285774,0.166667,0.166667,...,0.244949,0.244949,0.204124,0.240192,0.240192,0.204124,0.200160,0.163299,0.163299,0.122474
2,0.408248,0.408248,1.000000,0.204124,0.240000,0.320000,0.285774,0.320000,0.244949,0.244949,...,0.240000,0.240000,0.240000,0.274563,0.274563,0.240000,0.353009,0.320000,0.320000,0.280000
3,0.166667,0.166667,0.204124,1.000000,0.489898,0.285774,0.250000,0.244949,0.458333,0.458333,...,0.122474,0.122474,0.122474,0.160128,0.160128,0.122474,0.160128,0.122474,0.163299,0.122474
4,0.163299,0.163299,0.240000,0.489898,1.000000,0.440000,0.367423,0.400000,0.449073,0.408248,...,0.160000,0.160000,0.160000,0.196116,0.196116,0.160000,0.196116,0.160000,0.200000,0.200000


In [82]:
final_mod.insert(loc=0 ,column='models',value=cl1)

In [83]:
final_mod.head()

,models,iPad Pro 12.9,iPad Pro 10.5,iPad 9.7,Honor 9,Y7 Prime,MediaPad M3 Lite 8,MediaPad T3 10,MediaPad M3 Lite 10,nova 2 plus,...,Galaxy Note Pro 12.2 LTE,Galaxy Note Pro 12.2 3G,Galaxy Note Pro 12.2,Galaxy Tab Pro 12.2 LTE,Galaxy Tab Pro 12.2 3G,Galaxy Tab Pro 12.2,Galaxy Tab Pro 10.1 LTE,Galaxy Tab Pro 10.1,Galaxy Tab Pro 8.4 3G/LTE,Galaxy Tab Pro 8.4
0,iPad Pro 12.9,1.000000,0.791667,0.408248,0.166667,0.163299,0.244949,0.208333,0.285774,0.166667,...,0.244949,0.244949,0.204124,0.240192,0.240192,0.204124,0.200160,0.163299,0.163299,0.122474
1,iPad Pro 10.5,0.791667,1.000000,0.408248,0.166667,0.163299,0.244949,0.208333,0.285774,0.166667,...,0.244949,0.244949,0.204124,0.240192,0.240192,0.204124,0.200160,0.163299,0.163299,0.122474
2,iPad 9.7,0.408248,0.408248,1.000000,0.204124,0.240000,0.320000,0.285774,0.320000,0.244949,...,0.240000,0.240000,0.240000,0.274563,0.274563,0.240000,0.353009,0.320000,0.320000,0.280000
3,Honor 9,0.166667,0.166667,0.204124,1.000000,0.489898,0.285774,0.250000,0.244949,0.458333,...,0.122474,0.122474,0.122474,0.160128,0.160128,0.122474,0.160128,0.122474,0.163299,0.122474
4,Y7 Prime,0.163299,0.163299,0.240000,0.489898,1.000000,0.440000,0.367423,0.400000,0.449073,...,0.160000,0.160000,0.160000,0.196116,0.196116,0.160000,0.196116,0.160000,0.200000,0.200000


## Testing for a Device model

In [94]:
## input below the model

In [96]:
row1 = final_mod.loc[final_mod.models == 'Watch Series 1 Sport 42mm']

In [97]:
row1

,models,iPad Pro 12.9,iPad Pro 10.5,iPad 9.7,Honor 9,Y7 Prime,MediaPad M3 Lite 8,MediaPad T3 10,MediaPad M3 Lite 10,nova 2 plus,...,Galaxy Note Pro 12.2 LTE,Galaxy Note Pro 12.2 3G,Galaxy Note Pro 12.2,Galaxy Tab Pro 12.2 LTE,Galaxy Tab Pro 12.2 3G,Galaxy Tab Pro 12.2,Galaxy Tab Pro 10.1 LTE,Galaxy Tab Pro 10.1,Galaxy Tab Pro 8.4 3G/LTE,Galaxy Tab Pro 8.4
45,Watch Series 1 Sport 42mm,0.091287,0.091287,0.134164,0.136931,0.134164,0.134164,0.136931,0.134164,0.136931,...,0.089443,0.089443,0.268328,0.131559,0.131559,0.268328,0.131559,0.268328,0.134164,0.268328


## changing into dictionary to sort into descending order based on similarity score and producing top recommendations

In [98]:
row2 = row1.set_index('models')

In [99]:
df_dict = row2.to_dict('records')

In [100]:
sorted(df_dict[0] , key=df_dict[0].get, reverse = True)

['Watch Series 1 Sport 42mm',
 'Watch Series 2 Sport 42mm',
 'Watch Series 1 Sport 38mm',
 'Watch Series 2 42mm',
 'Watch Series 2 Sport 38mm',
 'Watch Edition Series 2 42mm',
 'Watch Edition Series 2 38mm',
 'Watch Series 2 38mm',
 'Watch Edition 42mm',
 'Watch 42mm',
 'Watch Sport 42mm',
 'Watch Sport 38mm',
 'Watch Edition 38mm',
 'Watch 38mm',
 'Gear 2 Neo',
 'Watch 2 Classic',
 'Gear S3 classic',
 'Gear S3 frontier',
 'Gear Live',
 'Gear 2',
 'Watch',
 'Gear S2 classic 3G',
 'Watch 2',
 'Gear S3 frontier LTE',
 'MediaPad T3 7.0',
 'Galaxy Tab 3 Lite 7.0 VE',
 'Gear S',
 'Honor Magic',
 'N1',
 'Galaxy Tab Active',
 'Galaxy Star 2 Plus',
 'Galaxy Star 2',
 'Galaxy Tab 3 Lite 7.0',
 'Enjoy 6s',
 'Enjoy 6',
 'Galaxy Tab 4 8.0 (2015)',
 'Galaxy Tab S 8.4',
 'Galaxy Tab S 10.5',
 'Galaxy Tab 4 8.0',
 'Galaxy Note Pro 12.2',
 'Galaxy Tab Pro 12.2',
 'Galaxy Tab Pro 10.1',
 'Galaxy Tab Pro 8.4',
 'Honor Holly 3',
 'Gear S3 classic LTE',
 'Galaxy Pocket 2',
 'iPhone SE',
 'P9',
 'Ascend Y2